In [105]:
import math
import numpy as np
# import os
# from session import session
# import requests
# headers = {"Cookie": "session=" + session}
# for n in range(1, 26):
#     print(n)
#     url = f"https://adventofcode.com/2023/day/{n}/input"
#     r = requests.get(url, headers=headers)
#     open(f"inputs23/{n}", "w").write(r.text)

# day 1

In [69]:
day1 = open("inputs23/1", "r").read()

total = 0
for l in day1.split("\n")[:-1]:
    first, last = None, None
    for c in l:
        if c.isnumeric():
            if first is None:
                first = c
            last = c
    total += int(first + last)
print(total)

nums = [
    "zero",
    "one",
    "two",
    "three",
    "four",
    "five",
    "six",
    "seven",
    "eight",
    "nine",
    "ten",
]
for i, n in enumerate(nums):
    # put the digit in the middle of the word -> easier to just surround it with its word for overlapping word situations
    day1 = day1.replace(n, f"{n}{i}{n}")
total = 0
for l in day1.split("\n")[:-1]:
    first, last = None, None
    for c in l:
        if c.isnumeric():
            if first is None:
                first = c
            last = c
    total += int(first + last)
print(total)

54953
53868


# day 2

In [99]:
day2 = open("inputs23/2", "r").read()

q1possible = (
    lambda d: (d.get("red", 0) <= 12)
    and (d.get("green", 0) <= 13)
    and (d.get("blue", 0) <= 14)
)


def fq2(lst):
    nd = {}
    for d in lst:
        for k, v in d.items():
            nd[k] = max(nd.get(k, 0), v)
    # return nd
    prod = 1
    for v in nd.values():
        prod *= v
    return prod


acc1 = 0
acc2 = 0
for l in day2.split("\n")[:-1]:
    p1, p2 = l.split(":")
    g = int(p1.split(" ")[1])
    sets = p2.split(";")
    items = [
        {ss.strip().split(" ")[1]: int(ss.strip().split(" ")[0]) for ss in s.split(",")}
        for s in sets
    ]
    q1 = all([q1possible(d) for d in items])
    acc1 += g if q1 else 0
    acc2 += fq2(items)
print(acc1)
print(acc2)

2727
56580


# day 3

In [337]:
day3 = open("inputs23/3", "r").read()

number_coords = []
part_numbers = []

arr = np.array([[c for c in l] for l in day3.split("\n")[:-1]])
for i, l in enumerate(day3.split("\n")[:-1]):
    j = 0
    while j < len(l):
        if l[j].isdecimal():
            coords = []
            j1 = j
            num = ""
            for e in l[j:]:
                if e.isdecimal():
                    num += e
                    coords.append((i, j))
                    j += 1
                else:
                    break
            j2 = j
            number = int(num)

            # Part 1
            omgeving = arr[
                max(0, i - 1) : min(len(arr), i + 2),
                max(0, j1 - 1) : min(len(l), j2 + 1),
            ]
            indicator = np.all(np.isin(omgeving, list(".0123456789")))
            if not indicator:
                part_numbers.append(number)

            # Part 2
            number_coords.append((number, coords))
        j += 1

print(sum(part_numbers))

# part 2
star_coords = np.argwhere(arr == "*")
star_neighbours = {}
for si, sj in star_coords:
    neighbours = set(
        (
            (max(0, si - 1), max(0, sj - 1)),
            (max(0, si - 1), sj),
            (max(0, si - 1), min(len(l), sj + 1)),
            (si, max(0, sj - 1)),
            (si, sj),
            (si, min(len(l), sj + 1)),
            (min(len(arr), si + 1), max(0, sj - 1)),
            (min(len(arr), si + 1), sj),
            (min(len(arr), si + 1), min(len(l), sj + 1)),
        )
    )

    for num, coords in number_coords:
        is_neighbour = any(c in neighbours for c in coords)
        if is_neighbour:
            star_neighbours[(si, sj)] = star_neighbours.get((si, sj), set()) | {
                (num, *coords)
            }

gear_ratios = 0
for i, d in star_neighbours.items():
    if len(d) == 2:
        dl = list(d)
        prod = dl[0][0] * dl[1][0]
        gear_ratios += prod
print(gear_ratios)

553079
84363105


# day 4

In [338]:
day4 = open("inputs23/4", "r").read()

p1sum = 0
cards = []
for l in day4.split("\n")[:-1]:
    p1, p2 = l.split(":")
    card = int(p1.split(" ")[-1])
    pw, pm = p2.split("|")
    wn = {int(i) for i in pw.strip().split(" ") if i.isdecimal()}
    mn = {int(i) for i in pm.strip().split(" ") if i.isdecimal()}
    matches = len(wn & mn)
    points = 2**matches // 2
    p1sum += points
    copy_count = sum(1 for c in cards if c == card)
    cards.append(card)
    copies = list(range(card+1, card+matches+1)) * (1+copy_count)
    cards += copies
print(p1sum)
print(len(cards))

32001
5037841


# day 5

https://adventofcode.com/2023/day/5

In [396]:
from typing import Callable


day5 = open("inputs23/5", "r").read()
day5 = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""

seeds = [int(i) for i in day5.split("\n")[0].split(": ")[1].split(" ")]
maps1 = [el.split(":")  for el in day5[:-1].split("\n\n")[1:]]  # :-1 to get rid of trailing \n
f = lambda s: [[int(i) for i in el.split(" ")] for el in s.split("\n")[1:]]
g = lambda m: {mn: f(mv) for mn, mv in m}
d = g(maps1[0:7])

def get_locator(lists: list[list[int]]) -> Callable[[int], int]:
    def f(e):
        for dst, src, ran in lists:
            if e in range(src, src+ran):
                new_e = e + (src - )

{'seed-to-soil map': [[50, 98, 2], [52, 50, 48]],
 'soil-to-fertilizer map': [[0, 15, 37], [37, 52, 2], [39, 0, 15]],
 'fertilizer-to-water map': [[49, 53, 8], [0, 11, 42], [42, 0, 7], [57, 7, 4]],
 'water-to-light map': [[88, 18, 7], [18, 25, 70]],
 'light-to-temperature map': [[45, 77, 23], [81, 45, 19], [68, 64, 13]],
 'temperature-to-humidity map': [[0, 69, 1], [1, 0, 69]],
 'humidity-to-location map': [[60, 56, 37], [56, 93, 4]]}